# Question by question

Alle Fragen werden einzeln und ohne weiteren Kontext (außer Systemprompts) gesendet

In [ ]:
import sys
sys.path.append('..')

import time
import survey.models 
from tqdm.notebook import tqdm
from survey.request import Request


# strategie laden
strategies = survey.models.StrategyModel()
strategies.load_strategies('questionbyquestion_%')
strategies_list = strategies.get_strategies()

# fragen laden
questions = survey.models.QuestionsModel()
questions_list = questions.get_questions()

# models laden
models = survey.models.ModelsModel()
models_list = models.get_models()

# request und response laden
run = survey.models.RunModel()
requests = Request()
response = survey.models.ResponseModel()

In [ ]:
print(f"Strategies: {len(strategies_list)} - {strategies_list}")
print(f"Questions: {len(questions_list)}")
print(f"Models: {len(models_list)}")

In [ ]:
for strategy in tqdm(strategies_list, desc="Strategies"):

    print(f"\n  Strategy: {strategy['name']}")

    # prompts lesen
    with open(f"../{strategy['system_path']}", "r") as f:
        system_prompt = f.read()

    with open(f"../{strategy['message_path']}", "r") as f:
        message_prompt = f.read()

    # models befragen
    for model_db_id, model_id in tqdm(models_list, desc=f"  Models ({strategy['name']})"):

        print(f"    {model_id}", end=" ", flush=True)

        prompt_tokens = 0
        completion_tokens = 0        
        start_time = time.time()

        run_id = run.write_run(model_db_id, strategy['id'])

        # einzelne fragen
        for question in tqdm(questions_list, desc="  Question"):

            question_id = question['id']
            question_text = question['text']

            message = message_prompt.replace("{question}", question_text)

            answer = requests.request_model(model_id, system_prompt, message, 'SINGLE')

            # tokens sammeln
            prompt_tokens += answer.usage.prompt_tokens
            completion_tokens += answer.usage.completion_tokens

            # DEBUG - Was kommt zurück?
            # print(f"\n=== Model: {model_id} ===")
            # print(answer.choices[0].message.content)
            # print("=" * 50)

            # antwort speichern
            try:
                response.write_respone(run_id, question_id, answer, 'SINGLE')
            except Exception as e:
                print(f"    --- Error {e} ---")

        # run updaten
        duration_time = time.time() - start_time
        run.update_run(prompt_tokens, completion_tokens, duration_time, run_id)
